# Download TF master set for Flagship (K562 only)

**Set environment**

In [1]:
source ../config/config_duke.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



## Extract command into a script

In [2]:
FDIRY=${FD_RES}/results/region/annotation_chipseq_tf_subset/accession_info_peaks.tsv
ls ${FDIRY}

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_chipseq_tf_subset/accession_info_peaks.tsv


In [3]:
FDIRY=${FD_RES}/results/region/annotation_chipseq_tf
FNAME=accession_info_peaks.tsv
FPATH=${FDIRY}/${FNAME}

cat  ${FPATH} | wc -l
echo
head ${FPATH}

736

Experiment_Accession	Peak_Accession	Biosample	Target	Lab	RFA	FN_INP	FN_OUT	URL	CMD
ENCSR805SIA	ENCFF500BWO	Homo sapiens K562 genetically modified (insertion) using CRISPR targeting H. sapiens PURB	PURB	Richard Myers, HAIB	ENCODE4	ENCFF500BWO.bed.gz	K562.ENCSR805SIA.ENCFF500BWO.PURB.bed.gz	https://www.encodeproject.org/ENCFF500BWO/@@download/ENCFF500BWO.bed.gz	wget https://www.encodeproject.org/ENCFF500BWO/@@download/ENCFF500BWO.bed.gz -O K562.ENCSR805SIA.ENCFF500BWO.PURB.bed.gz
ENCSR125RFR	ENCFF863ZFH	Homo sapiens K562 genetically modified (insertion) using CRISPR targeting H. sapiens ATF6	ATF6	Richard Myers, HAIB	ENCODE4	ENCFF863ZFH.bed.gz	K562.ENCSR125RFR.ENCFF863ZFH.ATF6.bed.gz	https://www.encodeproject.org/ENCFF863ZFH/@@download/ENCFF863ZFH.bed.gz	wget https://www.encodeproject.org/ENCFF863ZFH/@@download/ENCFF863ZFH.bed.gz -O K562.ENCSR125RFR.ENCFF863ZFH.ATF6.bed.gz
ENCSR841GLE	ENCFF515LWL	Homo sapiens K562 genetically modified (insertion) using CRISPR targeting H. sapiens ZNF

In [4]:
FD_INP=${FD_RES}/results/region/annotation_chipseq_tf
FN_INP=accession_info_peaks.tsv
FP_INP=${FD_INP}/${FN_INP}

FD_OUT=${FD_RES}/results/region/annotation_chipseq_tf
FN_OUT=download.sh
FP_OUT=${FD_OUT}/${FN_OUT}

cat  /dev/null     >  ${FP_OUT}
echo '#!/bin/bash' >> ${FP_OUT}

FP_AWK=/data/reddylab/software/bin/print_tab_cols.awk
cat ${FP_INP} \
| awk -f ${FP_AWK} -v cols=CMD \
>> ${FP_OUT}

In [5]:
cat  ${FP_OUT} | wc -l
echo
head ${FP_OUT}

736

#!/bin/bash
wget https://www.encodeproject.org/ENCFF500BWO/@@download/ENCFF500BWO.bed.gz -O K562.ENCSR805SIA.ENCFF500BWO.PURB.bed.gz 
wget https://www.encodeproject.org/ENCFF863ZFH/@@download/ENCFF863ZFH.bed.gz -O K562.ENCSR125RFR.ENCFF863ZFH.ATF6.bed.gz 
wget https://www.encodeproject.org/ENCFF515LWL/@@download/ENCFF515LWL.bed.gz -O K562.ENCSR841GLE.ENCFF515LWL.ZNF217.bed.gz 
wget https://www.encodeproject.org/ENCFF121HYT/@@download/ENCFF121HYT.bed.gz -O K562.ENCSR014ARU.ENCFF121HYT.ATF2.bed.gz 
wget https://www.encodeproject.org/ENCFF169QYL/@@download/ENCFF169QYL.bed.gz -O K562.ENCSR172XJS.ENCFF169QYL.ZNF165.bed.gz 
wget https://www.encodeproject.org/ENCFF528IDR/@@download/ENCFF528IDR.bed.gz -O K562.ENCSR676KDJ.ENCFF528IDR.ARID4B.bed.gz 
wget https://www.encodeproject.org/ENCFF303CVC/@@download/ENCFF303CVC.bed.gz -O K562.ENCSR817FMN.ENCFF303CVC.TEAD4.bed.gz 
wget https://www.encodeproject.org/ENCFF295XBK/@@download/ENCFF295XBK.bed.gz -O K562.ENCSR845GGY.ENCFF295XBK.ZNF263.bed.gz

## Download files and sort

In [6]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 8G \
    --output ${FD_LOG}/download_chipseq_tf_full.%a.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories
source ../config/config_duke.sh

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set I/O 
FDIRY=${FD_RES}/results/region/annotation_chipseq_tf
FNAME=download.sh
FPATH=${FDIRY}/${FNAME}

### execute: download
echo "Downloading files..."

chmod +x ${FPATH}
cd   ${FDIRY}
bash ${FPATH}

### execute: sort
echo "Sorting files..."

FD_INP=${FD_RES}/results/region/annotation_chipseq_tf
FP_INPS=($(ls ${FD_INP}/K562.*.bed.gz))
FP_TMP=${FD_INP}/tmp.bed.gz

for FP_INP in ${FP_INPS[@]}; do

    ### show progress
    FN_INP=$(basename ${FP_INP})
    echo ${FN_INP}

    ### sort bed file
    zcat ${FP_INP} | sort -k 1,1 -k2,2n | gzip -c > ${FP_TMP}
    mv ${FP_TMP} ${FP_INP}
    
    ### check if sorted
    zcat ${FP_INP} | sort -k 1,1 -k2,2n -C || echo -n "not" ; echo "sorted!"
    
done

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29696139


**Check results**

In [8]:
FD_INP=${FD_RES}/results/region/annotation_chipseq_tf
FP_INPS=($(ls ${FD_INP}/K562.*.bed.gz))

echo ${#FP_INPS[@]}

735
